In [ ]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np

In [4]:

if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # drive.mount('/content/drive')
  proj_dir = "/content/drive/MyDrive/ece884_project/"
else:
  proj_dir = ""

Data fields

- id - a unique identifier for each trip
- vendor_id - a code indicating the provider associated with the trip record
- pickup_datetime - date and time when the meter was engaged
- dropoff_datetime - date and time when the meter was disengaged
- passenger_count - the number of passengers in the vehicle (driver entered value)
- pickup_longitude - the longitude where the meter was engaged
- pickup_latitude - the latitude where the meter was engaged
- dropoff_longitude - the longitude where the meter was disengaged
- dropoff_latitude - the latitude where the meter was disengaged 
- store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a - connection to the server - Y=store and forward; N=not a store and forward trip
- trip_duration - duration of the trip in seconds

In [5]:
df = pd.read_csv(f"{proj_dir}misc_data/taxi/taxi.csv", low_memory=False)

In [6]:
df["pickup_datetime"] = pd.to_datetime(df.pickup_datetime, format='%Y-%m-%d %H:%M:%S')
df["dropoff_datetime"] = pd.to_datetime(df.dropoff_datetime, format='%Y-%m-%d %H:%M:%S')

In [7]:
def create_features(data, col):
    data['dayofweek'] = data[col].dt.dayofweek
    data['month'] = data[col].dt.month
    data['year'] = data[col].dt.year
    data['quarter'] = data[col].dt.quarter
    data['dayofyear'] = data[col].dt.dayofyear
    data['weekofyear'] = data[col].dt.isocalendar().week
    data['dayofmonth'] = data[col].dt.day
    data['hour'] = data[col].dt.hour
    data['minute'] = data[col].dt.minute
    data['second'] = data[col].dt.second
    data['horizon'] = (data[col] - data[col].min()) / np.timedelta64(1, 's')

    X = data[['year','quarter','month','dayofweek',
           'dayofyear','dayofmonth','weekofyear',
            'hour', 'minute', 'second', 'horizon']]

    X.columns = [str(x) + "_" + col for x in X.columns]
    
    return X

In [8]:
taxi_data = pd.concat([create_features(df, 'pickup_datetime'),
           create_features(df, 'dropoff_datetime'),
           df[["passenger_count",	"pickup_longitude",	"pickup_latitude",	"dropoff_longitude",	"dropoff_latitude",	"trip_duration"]]], axis=1)

In [9]:
taxi_data.shape

(1458644, 28)

In [ ]:
taxi_data.to_csv(f"{proj_dir}data_clean/taxi.csv", index=False)